In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [5]:
pre_trained_model = InceptionV3(input_shape=(150,150,3),include_top=False,weights=None)
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop_2.h5'
pre_trained_model.load_weights(local_weights_file)

In [6]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [8]:
#pre_trained_model.summary()

In [12]:
last_layer = pre_trained_model.get_layer('mixed7')

last_output = last_layer.output

print(last_output)

Tensor("mixed7_2/Identity:0", shape=(None, 7, 7, 768), dtype=float32)


In [18]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1,activation='sigmoid')(x)

model = Model(pre_trained_model.input,x)
model.compile(optimizer=RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,
                                   height_shift_range=0.2,horizontal_flip=True,shear_range=0.2,zoom_range=0.2)
valid_datagen = ImageDataGenerator(rescale=1/255)

train_dir = "cats_and_dogs_filtered/train"
val_dir ="cats_and_dogs_filtered/validation"

train_generator = train_datagen.flow_from_directory(train_dir,batch_size=20,target_size=(150,150),class_mode='binary')
val_generator = valid_datagen.flow_from_directory(val_dir,batch_size=20,target_size=(150,150),class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [20]:
history = model.fit_generator(train_generator,validation_data=val_generator,epochs=3,steps_per_epoch=100,validation_steps=50)

Epoch 1/3
100/100 [==============================] - 447s 4s/step - loss: 0.4876 - accuracy: 0.7610 - val_loss: 0.2104 - val_accuracy: 0.9350
Epoch 2/3
100/100 [==============================] - 450s 4s/step - loss: 0.3574 - accuracy: 0.8420 - val_loss: 0.2611 - val_accuracy: 0.9380
Epoch 3/3
100/100 [==============================] - 430s 4s/step - loss: 0.3427 - accuracy: 0.8460 - val_loss: 0.3109 - val_accuracy: 0.9340
